In [1]:
import torchvision
torchvision.__version__

'0.6.0a0+82fd1c8'

In [2]:
import os
import pandas as pd
import matplotlib.pylab as plt
import torch
%matplotlib inline

In [3]:
def scale_label(a,b):
    div = [ai/bi for ai,bi in zip(a,b)]
    return div

def rescale_label(a,b):
    div = [ai*bi for ai,bi in zip(a,b)]
    return div

In [4]:
import torchvision.transforms.functional as TF

def resize_img_label(image, label = (0., 0.), target_size=(256,256)):
    w_orig, h_orig = image.size
    w_target, h_target = target_size
    cx, cy = label
    
    # resize image and label
    image_new = TF.resize(image, target_size)
    label_new = cx/w_orig*w_target, cy/h_orig*h_target
    
    return image_new, label_new

In [5]:
def transformer(image, label, params):
    image, label = resize_img_label(image,label,params["target_size"])
    
    if params["scale_label"]:
        label=scale_label(label, params["target_size"])
        
    image = TF.to_tensor(image)
    return image, label

In [6]:
from torch.utils.data import Dataset
from PIL import Image

In [7]:
class AMD_dataset(Dataset):
    def __init__(self, path2data, transform, trans_params):
        pass
    
    def __len__(self):
        # return size of dataset
        return len(self.labels)
    
    def __getitem__(self, idx):
        pass

In [8]:
def __init__(self, path2data, transform, trans_params):
    
    # full path of the labels file
    path2labels = os.path.join(path2data, "Training400", "Fovea_location.xlsx")
    
    # read labels as a data frame
    labels_df = pd.read_excel(path2labels,index_col="ID")
    
    # extract labels
    self.labels = labels_df[["Fovea_X", "Fovea_Y"]].values
    
    # extract ID and imgName columns
    self.imgName = labels_df["imgName"]
    self.ids = labels_df.index
    
    self.fullPath2img = [0]*len(self.ids)
    for id_ in self.ids:
        if self.imgName[id_][0] == "A":
            prefix = "AMD"
        else:
            prefix = "Non-AMD"
        
        self.fullPath2img[id_-1] = os.path.join(path2data, "Training400", prefix, self.imgName[id_])
    
    self.transform = transform
    self.trans_params = trans_params

In [11]:
def __getitem__(self, idx):
    # load PIL image
    image = Image.open(self.fullPath2img[idx])
    label = self.labels[idx]
    
    # transform to tensor
    image, label = self.transform(image, label, self.trans_params)
    
    return image, label

In [12]:
AMD_dataset.__init__ = __init__
AMD_dataset.__getitem__ = __getitem__

In [13]:
path2data = "./data/"
trans_params_val = {
    "target_size": (256, 256),
    "p_hflip": 0.0,
    "p_vflip": 0.0,
    "p_shift": 0.0,
    "p_brightness": 0.0,
    "p_contrast": 0.0,
    "p_gamma": 0.0,
    "gamma": 0.0,
    "scale_label": True,
}
amd_ds2 = AMD_dataset(path2data, transformer, trans_params_val)

In [15]:
from sklearn.model_selection import ShuffleSplit

sss = ShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

indices = range(len(amd_ds2))
for train_index, val_index in sss.split(indices):
    print(len(train_index))
    print("-"*10)
    print(len(val_index))

320
----------
80


In [16]:
from torch.utils.data import Subset
val_ds = Subset(amd_ds2, val_index)
print(len(val_ds))

80


In [17]:
from torch.utils.data import DataLoader
val_dl = DataLoader(val_ds, batch_size=16, shuffle=False)

In [18]:
import torch.nn as nn
import torch.nn.functional as F

In [19]:
class Net(nn.Module):
    def __init__(self, params):
        super(Net, self).__init__()
    
    def forward(self, x):
        return x

In [24]:
def __init__(self, params):
    super(Net, self).__init__()
    
    C_in, H_in, W_in = params["input_shape"]
    init_f = params["initial_filters"]
    num_outputs = params["num_outputs"]
    
    self.conv1 = nn.Conv2d(C_in, init_f, kernel_size=3, stride=2, padding=1)
    self.conv2 = nn.Conv2d(init_f + C_in, 2 * init_f, kernel_size=2, stride=1, padding=1)
    self.conv3 = nn.Conv2d(3 * init_f + C_in, 4 * init_f, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(7 * init_f + C_in, 8 * init_f, kernel_size=3, padding=1)
    self.conv5 = nn.Conv2d(15 * init_f + C_in, 16 * init_f, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(16 * init_f, num_outputs)

In [25]:
def forward(self, x):
    identity = F.avg_pool2d(x, 4, 4)
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, 2, 2)
    x = torch.cat((x, identity), dim=1)
    
    identity = F.avg_pool2d(x, 2, 2)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, 2, 2)
    x = torch.cat((x, identity), dim=1)
    
    identity = F.avg_pool2d(x, 2, 2)
    x = F.relu(self.conv3(x))
    x = F.maxpool2d(x, 2, 2)
    x = torch.cat((x, identity), dim=1)
    
    identity = F.avg_pool2d(x, 2, 2)
    x = F.relu(self.conv4(x))
    x = F.max_pool2d(x, 2, 2)
    x = torch.cat((x, identity), dim=1)
    
    x = F.relu(self.conv5(x))
    
    x = F.adaptive_avg_pool2d(x, 1)
    x = x.reshape(x.size(0), -1)
    
    x = self.fc1(x)
    return x

In [26]:
Net.__init__ = __init__
Net.forward = forward

In [27]:
params_model = {
    "input_shape": (3, 256, 256),
    "initial_filters": 16,
    "num_outputs": 2,
}

# create model
model = Net(params_model)
model.eval()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(19, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(51, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(115, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(243, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=256, out_features=2, bias=True)
)

In [29]:
# move the model to cuda/gpu device
if torch.cuda.is_available():
    device = torch.device("cuda")
    model=model.to(device)